In [1]:
""" IMPORT NEEDED MODULES """

import pandas as pd
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import cmocean
import cmocean.cm as cmo

import torch
import torch.utils.data

import re
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler

import time as t
import torch.optim as optim
from torch.autograd import Variable
from torchvision import transforms
from torch.utils.data.dataset import Dataset  # For custom datasets

In [2]:
startingDir = os.getcwd()
print(startingDir)

C:\Users\Peter\Desktop\Thermo_R_Code\FinalThermo\new\BCI-ML


In [24]:
"""
# Takes in a string that represents name of csv file and 
# converts that string to one that matches the date string present
# in the old master file (old master file refers to the raw excel file
# filled with extraneous data from BCI)that contains the date/times 
# for recorded GPP values
Description pulled from make-master-non-c0ontinuous

# takes a file name in the form YYYY_MM_DD_HH_MM_bci_t and returns new date in form 2/17/2015 15:30. 
# with times not at 30 minute intervals rounded to the nearest 30 min

"""

# takes a file name in the form YYYY_MM_DD_HH_MM_bci_t and returns new date in form 2/17/2015 15:30. 
# with times not at 30 minute intervals rounded to the nearest 30 min
def getRoundedTime(inTime):
    minDivFiveFlag = True
    year = inTime[2:4]
    month = inTime[5:7]
    day = inTime[8:10]
    hour = inTime[11:13]
    minuteInt = int(inTime[14:16])
    # Round minute such that those 0-15 and 45-59 get rounded to 0 and all other values to 30
    minuteStr = "XX"
    if(minuteInt <= 15):
        minuteStr = "00"
    elif(minuteInt >= 45):
        minuteStr = "00"
        hour = str(int(hour)+1)
    else:
        minuteStr = "30"
    second = "00"
    if(int(hour) == 0):
        hour = "0"
    elif(int(hour) < 10):
        hour = hour[1:2]
    # No leading zeroes in day or month
    if(int(month) < 10):
        month = month[1:2]
    if(int(day) < 10):
        day = day[1:2]
    result = month + "/" + day + "/" + year + " " + hour + ":" + minuteStr #+ ":" + second + " " + timeDay
    return result

In [4]:
""" Takes in a date in form 2/17/2015 15:30 and another date in the same form, checks if times are the same"""
def timeMatch(inTimeOne, inTimeTwo):
    timeOne = inTimeOne[inTimeOne.index(" "):]
    timeTwo = inTimeTwo[inTimeTwo.index(" "):]
    if(timeOne == timeTwo):
        return True
    else:
        return False

print(timeMatch("9/30/2315 15:00", "9/30/15 15:00"))

True


In [5]:
csvfilesMLLocation = r"C:/Users/Peter/Desktop/Thermo_R_Code/FinalThermo/new2/csvdataFULL/BCIML_Dataset"
masterfileMLLocation = r"C:/Users/Peter/Desktop/Thermo_R_Code/FinalThermo/new2/csvdataFULL/BCIML_Dataset/populatedGPPsFULL.csv"
testData = r"C:/Users/Peter/Desktop/Thermo_R_Code/FinalThermo/new/BCI-ML/Monthly_Data_Summaries_csv/DAYTIME_new9930834___/test_data_DAYTIME_13-446994955314361.csv"
analysisLocation = r"C:/Users/Peter/Desktop/Thermo_R_Code/FinalThermo/new2/tempplots"
all_working = r"C:/Users/Peter/Desktop/Thermo_R_Code/FinalThermo/new2/all-30min-Apr2018_working.csv"

# Get data from test data, where we store targets and predictions of network
testDataInfo = pd.read_csv(testData)
testDataFileNames = testDataInfo['filename']
allInfo = pd.read_csv(all_working)
par_vals = allInfo['PAR']
date_vals = allInfo['date']

In [6]:
# Each data frame holds PAR and corresponding date values, each split into individual months
#
par_date_dic = {"PAR": par_vals, "date": date_vals}
full_df = pd.concat(par_date_dic,axis=1)
regex_time_range = re.compile("2015_.._.._(0[4-9]_..|1[0-9]_..|20_00)_bci_t")

# Filter out time space we want ####################################
#with contains and regex abo e

print(full_df)


#https://stackoverflow.com/questions/27975069/how-to-filter-rows-containing-a-string-pattern-from-a-pandas-dataframe
feb_df = full_df[full_df['date'].str.startswith('2/')]
mar_df = full_df[full_df['date'].str.startswith('3/')]
apr_df = full_df[full_df['date'].str.startswith('4/')]
may_df = full_df[full_df['date'].str.startswith('5/')]
jun_df = full_df[full_df['date'].str.startswith('6/')]
jul_df = full_df[full_df['date'].str.startswith('7/')]
aug_df = full_df[full_df['date'].str.startswith('8/')]
sep_df = full_df[full_df['date'].str.startswith('9/')]

# save data frames
par_csv_save_location = r"C:/Users/Peter/Desktop/Thermo_R_Code/FinalThermo/new/BCI-ML/Images and Plots/PAR Data"
os.chdir(par_csv_save_location)
feb_df.to_csv('feb_par.csv')
mar_df.to_csv('mar_par.csv')
apr_df.to_csv('apr_par.csv')
may_df.to_csv('may_par.csv')
jun_df.to_csv('jun_par.csv')
jul_df.to_csv('jul_par.csv')
aug_df.to_csv('aug_par.csv')
sep_df.to_csv('sep_par.csv')
full_df.to_csv('full_par.csv')

              PAR           date
0     2043.833333  2/17/15 13:00
1     1013.350000  2/17/15 13:30
2      804.966667  2/17/15 14:00
3      658.200000  2/17/15 14:30
4      424.350000  2/17/15 15:00
...           ...            ...
8363     0.340000  9/30/15 21:30
8364     0.340000  9/30/15 22:00
8365     0.340167  9/30/15 22:30
8366     0.340000  9/30/15 23:00
8367     0.340000  9/30/15 23:30

[8368 rows x 2 columns]


In [30]:
"""Goes through a list of filenames, rounding their time values to the nearest 30 min interval,
and then looking at PAR values from the mastermaster file (all-30min-Apr2018_working) for the 
same times but +/- 6 days apart (THIS CAN BE CHANGED, MARKED WITH #XKDCD.) 
The average, median, standard deviation and range is then pulled form these
14 days of PAR data. Each of these values is saved, along with the exact time PAR value, and the 
difference between these two data points"""
os.chdir(csvfilesMLLocation)
sourceFile = "populatedGPPsDAYTIME.csv"
daytimePopulatedInfo =pd.read_csv(sourceFile)
filenames = daytimePopulatedInfo["FILE"]

savedIndex = 0
# Holds individual rows of table, will be concatted
fullInfoList = list()
resultCsvFileName = "PAR_Analysis_" + testData[testData.rfind('/')+1:testData.rfind('.')] + ".csv"
resultTextFileName = "Par_Analysis_textlog_" + testData[testData.rfind('/')+1:testData.rfind('.')] +".txt"


savedIndex = 0
for filename in filenames:
    newFilename = getRoundedTime(filename)
    for i, r in full_df.iloc[savedIndex:].iterrows():
        # Go through each row where last left off, looking for matching time
        if(r['date']==newFilename):
            print(newFilename)
            savedIndex = i
            statList = list()
            backIdx = i-1
            forwardIdx = i+1
            statList.append(full_df.iloc[i]["PAR"])
            #XCKD, can change range of days pulled from here by changing size of list
            # where (size of list - 1) / 2 = number of days in each direction
            while(len(statList) < 13):
                # Start iterating backwards and forwards, adding PAR values to stat list as we go
                # foward and backindex will stop if they reach endpoints, and the other will grow the list
                # if both endpoints are reached, we stop.
                if(backIdx > -1):
                    if(timeMatch(newFilename, full_df.iloc[backIdx]['date'])):
                        statList.append(full_df.iloc[backIdx]['PAR'])
                if(forwardIdx < len(full_df)):
                    if(timeMatch(newFilename, full_df.iloc[forwardIdx]['date'])):
                        statList.append(full_df.iloc[forwardIdx]['PAR'])
                backIdx = backIdx - 1
                forwardIdx = forwardIdx + 1
                if(forwardIdx >= len(full_df) and backIdx <= -1):
                    break
            # If we can find no similar times to take average PAR from then log this in text file and move on
            if(len(statList) <= 1):
                with open(resultTextFileName, 'a') as out:
                    out.write(newFilename + ' has no additional par to take average of...')
                    break
            statSeries = pd.Series(statList)
            descSeries = statSeries.describe()
            detailsSeries = pd.Series({'newFilename': newFilename,'PAR': full_df.iloc[i]["PAR"], 'PARDIFF':abs(full_df.iloc[i]["PAR"]-descSeries['mean'])})
            finalInfoSeries = pd.concat([detailsSeries, descSeries])
            fullInfoList.append(finalInfoSeries)
            break

newSeries = (pd.concat(fullInfoList, axis=1)).transpose()
print("ASDASDASD")
print(newSeries)
print(type(newSeries))
newSeries.to_csv(par_csv_save_location+"/"+resultCsvFileName)
        

3/14/15 13:30
3/14/15 13:30
3/14/15 13:30
3/14/15 14:00
3/14/15 14:00
3/14/15 14:00
3/14/15 14:00
3/14/15 14:00
3/14/15 14:00
3/14/15 14:00
3/14/15 14:30
3/14/15 14:30
3/14/15 14:30
3/14/15 14:30
3/14/15 14:30
3/14/15 15:00
3/14/15 15:00
3/14/15 15:00
3/14/15 15:00
3/14/15 15:00
3/14/15 15:00
3/14/15 15:00
3/14/15 15:30
3/14/15 15:30
3/14/15 15:30
3/14/15 15:30
3/14/15 15:30
3/14/15 16:00
3/14/15 16:00
3/14/15 16:00
3/14/15 16:00
3/14/15 16:00
3/14/15 16:00
3/14/15 16:00
3/14/15 16:30
3/14/15 16:30
3/14/15 16:30
3/14/15 16:30
3/14/15 16:30
3/14/15 17:00
3/14/15 17:00
3/14/15 17:00
3/14/15 17:00
3/14/15 17:00
3/14/15 17:00
3/14/15 17:00
3/14/15 17:30
3/14/15 17:30
3/14/15 17:30
3/14/15 17:30
3/14/15 17:30
3/14/15 18:00
3/14/15 18:00
3/14/15 18:00
3/14/15 18:00
3/14/15 18:00
3/14/15 18:00
3/14/15 18:00
3/14/15 18:30
3/14/15 18:30
3/14/15 18:30
3/14/15 18:30
3/14/15 18:30
3/14/15 19:00
3/14/15 19:00
3/14/15 19:00
3/14/15 19:00
3/14/15 19:00
3/14/15 19:00
3/14/15 19:00
3/14/15 19:30
3/14/1

3/17/15 19:00
3/17/15 19:00
3/17/15 19:00
3/17/15 19:00
3/17/15 19:00
3/17/15 19:30
3/17/15 19:30
3/17/15 19:30
3/17/15 19:30
3/17/15 19:30
3/17/15 20:00
3/17/15 20:00
3/17/15 20:00
3/18/15 4:00
3/18/15 4:00
3/18/15 4:00
3/18/15 4:00
3/18/15 4:30
3/18/15 4:30
3/18/15 4:30
3/18/15 4:30
3/18/15 4:30
3/18/15 5:00
3/18/15 5:00
3/18/15 5:00
3/18/15 5:00
3/18/15 5:30
3/18/15 5:30
3/18/15 5:30
3/18/15 5:30
3/18/15 5:30
3/18/15 6:00
3/18/15 6:00
3/18/15 6:00
3/18/15 6:00
3/18/15 6:30
3/18/15 6:30
3/18/15 6:30
3/18/15 6:30
3/18/15 6:30
3/18/15 7:00
3/18/15 7:00
3/18/15 7:00
3/18/15 7:00
3/18/15 7:30
3/18/15 7:30
3/18/15 7:30
3/18/15 7:30
3/18/15 7:30
3/18/15 8:00
3/18/15 8:00
3/18/15 8:00
3/18/15 8:00
3/18/15 8:30
3/18/15 8:30
3/18/15 8:30
3/18/15 8:30
3/18/15 8:30
3/18/15 9:00
3/18/15 9:00
3/18/15 9:00
3/18/15 9:00
3/18/15 9:30
3/18/15 9:30
3/18/15 9:30
3/18/15 9:30
3/18/15 9:30
3/18/15 10:00
3/18/15 10:00
3/18/15 10:00
3/18/15 10:00
3/18/15 10:00
3/18/15 10:00
3/18/15 10:00
3/18/15 10:30
3/18

3/21/15 10:00
3/21/15 10:00
3/21/15 10:00
3/21/15 10:00
3/21/15 10:00
3/21/15 10:30
3/21/15 10:30
3/21/15 10:30
3/21/15 10:30
3/21/15 10:30
3/21/15 11:00
3/21/15 11:00
3/21/15 11:00
3/21/15 11:00
3/21/15 11:00
3/21/15 11:00
3/21/15 11:00
3/21/15 11:30
3/21/15 11:30
3/21/15 11:30
3/21/15 11:30
3/21/15 11:30
3/21/15 12:00
3/21/15 12:00
3/21/15 12:00
3/21/15 12:00
3/21/15 12:00
3/21/15 12:00
3/21/15 12:00
3/21/15 12:30
3/21/15 12:30
3/21/15 12:30
3/21/15 12:30
3/21/15 12:30
3/21/15 13:00
3/21/15 13:00
3/21/15 13:00
3/21/15 13:00
3/21/15 13:00
3/21/15 13:00
3/21/15 13:00
3/21/15 13:30
3/21/15 13:30
3/21/15 13:30
3/21/15 13:30
3/21/15 13:30
3/21/15 14:00
3/21/15 14:00
3/21/15 14:00
3/21/15 14:00
3/21/15 14:00
3/21/15 14:00
3/21/15 14:00
3/21/15 14:30
3/21/15 14:30
3/21/15 14:30
3/21/15 14:30
3/21/15 14:30
3/21/15 15:00
3/21/15 15:00
3/21/15 15:00
3/21/15 15:00
3/21/15 15:00
3/21/15 15:00
3/21/15 15:00
3/21/15 15:30
3/21/15 15:30
3/21/15 15:30
3/21/15 15:30
3/21/15 15:30
3/21/15 16:00
3/21/1

3/24/15 15:30
3/24/15 15:30
3/24/15 15:30
3/24/15 15:30
3/24/15 15:30
3/24/15 16:00
3/24/15 16:00
3/24/15 16:00
3/24/15 16:00
3/24/15 16:00
3/24/15 16:00
3/24/15 16:00
3/24/15 16:30
3/24/15 16:30
3/24/15 16:30
3/24/15 16:30
3/24/15 16:30
3/24/15 17:00
3/24/15 17:00
3/24/15 17:00
3/24/15 17:00
3/24/15 17:00
3/24/15 17:00
3/24/15 17:00
3/24/15 17:30
3/24/15 17:30
3/24/15 17:30
3/24/15 17:30
3/24/15 17:30
3/24/15 18:00
3/24/15 18:00
3/24/15 18:00
3/24/15 18:00
3/24/15 18:00
3/24/15 18:00
3/24/15 18:00
3/24/15 18:30
3/24/15 18:30
3/24/15 18:30
3/24/15 18:30
3/24/15 18:30
3/24/15 19:00
3/24/15 19:00
3/24/15 19:00
3/24/15 19:00
3/24/15 19:00
3/24/15 19:00
3/24/15 19:00
3/24/15 19:30
3/24/15 19:30
3/24/15 19:30
3/24/15 19:30
3/24/15 19:30
3/24/15 20:00
3/24/15 20:00
3/24/15 20:00
3/25/15 4:00
3/25/15 4:00
3/25/15 4:00
3/25/15 4:00
3/25/15 4:30
3/25/15 4:30
3/25/15 4:30
3/25/15 4:30
3/25/15 4:30
3/25/15 5:00
3/25/15 5:00
3/25/15 5:00
3/25/15 5:00
3/25/15 5:30
3/25/15 5:30
3/25/15 5:30
3/25/15 

3/28/15 5:00
3/28/15 5:00
3/28/15 5:00
3/28/15 5:30
3/28/15 5:30
3/28/15 5:30
3/28/15 5:30
3/28/15 5:30
3/28/15 6:00
3/28/15 6:00
3/28/15 6:00
3/28/15 6:00
3/28/15 6:30
3/28/15 6:30
3/28/15 6:30
3/28/15 6:30
3/28/15 6:30
3/28/15 7:00
3/28/15 7:00
3/28/15 7:00
3/28/15 7:00
3/28/15 7:30
3/28/15 7:30
3/28/15 7:30
3/28/15 7:30
3/28/15 7:30
3/28/15 8:00
3/28/15 8:00
3/28/15 8:00
3/28/15 8:00
3/28/15 8:30
3/28/15 8:30
3/28/15 8:30
3/28/15 8:30
3/28/15 8:30
3/28/15 9:00
3/28/15 9:00
3/28/15 9:00
3/28/15 9:00
3/28/15 9:30
3/28/15 9:30
3/28/15 9:30
3/28/15 9:30
3/28/15 9:30
3/28/15 10:00
3/28/15 10:00
3/28/15 10:00
3/28/15 10:00
3/28/15 10:00
3/28/15 10:00
3/28/15 10:00
3/28/15 10:30
3/28/15 10:30
3/28/15 10:30
3/28/15 10:30
3/28/15 10:30
3/28/15 11:00
3/28/15 11:00
3/28/15 11:00
3/28/15 11:00
3/28/15 11:00
3/28/15 11:00
3/28/15 11:00
3/28/15 11:30
3/28/15 11:30
3/28/15 11:30
3/28/15 11:30
3/28/15 11:30
3/28/15 12:00
3/28/15 12:00
3/28/15 12:00
3/28/15 12:00
3/28/15 12:00
3/28/15 12:00
3/28/15 

...

9/11/15 12:00
9/11/15 12:00
9/11/15 12:00
9/11/15 12:00
9/11/15 12:00
9/11/15 12:00
9/11/15 12:00
9/11/15 12:30
9/11/15 12:30
9/11/15 12:30
9/11/15 12:30
9/11/15 12:30
9/11/15 13:00
9/11/15 13:00
9/11/15 13:00
9/11/15 13:00
9/11/15 13:00
9/11/15 13:00
9/11/15 13:00
9/11/15 13:30
9/11/15 13:30
9/11/15 13:30
9/11/15 13:30
9/11/15 13:30
9/11/15 14:00
9/11/15 14:00
9/11/15 14:00
9/11/15 14:00
9/11/15 14:00
9/11/15 14:00
9/11/15 14:00
9/11/15 14:30
9/11/15 14:30
9/11/15 14:30
9/11/15 14:30
9/11/15 14:30
9/11/15 15:00
9/11/15 15:00
9/11/15 15:00
9/11/15 15:00
9/11/15 15:00
9/11/15 15:00
9/11/15 15:00
9/11/15 15:30
9/11/15 15:30
9/11/15 15:30
9/11/15 15:30
9/11/15 15:30
9/11/15 16:00
9/11/15 16:00
9/11/15 16:00
9/11/15 16:00
9/11/15 16:00
9/11/15 16:00
9/11/15 16:00
9/11/15 16:30
9/11/15 16:30
9/11/15 16:30
9/11/15 16:30
9/11/15 16:30
9/11/15 17:00
9/11/15 17:00
9/11/15 17:00
9/11/15 17:00
9/11/15 17:00
9/11/15 17:00
9/11/15 17:00
9/11/15 17:30
9/11/15 17:30
9/11/15 17:30
9/11/15 17:30
9/11/1

9/20/15 9:00
9/20/15 9:00
9/20/15 9:30
9/20/15 9:30
9/20/15 9:30
9/20/15 9:30
9/20/15 9:30
9/20/15 10:00
9/20/15 10:00
9/20/15 10:00
9/20/15 10:00
9/20/15 10:00
9/20/15 10:00
9/20/15 10:00
9/20/15 10:30
9/20/15 10:30
9/20/15 10:30
9/20/15 10:30
9/20/15 10:30
9/20/15 11:00
9/20/15 11:00
9/20/15 11:00
9/20/15 11:00
9/20/15 11:00
9/20/15 11:00
9/20/15 11:00
9/20/15 11:30
9/20/15 11:30
9/20/15 11:30
9/20/15 11:30
9/20/15 11:30
9/20/15 12:00
9/20/15 12:00
9/20/15 12:00
9/20/15 12:00
9/20/15 12:00
9/20/15 12:00
9/20/15 12:00
9/20/15 12:30
9/20/15 12:30
9/20/15 12:30
9/20/15 12:30
9/20/15 12:30
9/20/15 13:00
9/20/15 13:00
9/20/15 13:00
9/20/15 13:00
9/20/15 13:00
9/20/15 13:00
9/20/15 13:00
9/20/15 13:30
9/20/15 13:30
9/20/15 13:30
9/20/15 13:30
9/20/15 13:30
9/20/15 14:00
9/20/15 14:00
9/20/15 14:00
9/20/15 14:00
9/20/15 14:00
9/20/15 14:00
9/20/15 14:00
9/20/15 14:30
9/20/15 14:30
9/20/15 14:30
9/20/15 14:30
9/20/15 14:30
9/20/15 15:00
9/20/15 15:00
9/20/15 15:00
9/20/15 15:00
9/20/15 15:00

9/23/15 14:30
9/23/15 15:00
9/23/15 15:00
9/23/15 15:00
9/23/15 15:00
9/23/15 15:00
9/23/15 15:00
9/23/15 15:00
9/23/15 15:30
9/23/15 15:30
9/23/15 15:30
9/23/15 15:30
9/23/15 15:30
9/23/15 16:00
9/23/15 16:00
9/23/15 16:00
9/23/15 16:00
9/23/15 16:00
9/23/15 16:00
9/23/15 16:00
9/23/15 16:30
9/23/15 16:30
9/23/15 16:30
9/23/15 16:30
9/23/15 16:30
9/23/15 17:00
9/23/15 17:00
9/23/15 17:00
9/23/15 17:00
9/23/15 17:00
9/23/15 17:00
9/23/15 17:00
9/23/15 17:30
9/23/15 17:30
9/23/15 17:30
9/23/15 17:30
9/23/15 17:30
9/23/15 18:00
9/23/15 18:00
9/23/15 18:00
9/23/15 18:00
9/23/15 18:00
9/23/15 18:00
9/23/15 18:00
9/23/15 18:30
9/23/15 18:30
9/23/15 18:30
9/23/15 18:30
9/23/15 18:30
9/23/15 19:00
9/23/15 19:00
9/23/15 19:00
9/23/15 19:00
9/23/15 19:00
9/23/15 19:00
9/23/15 19:00
9/23/15 19:30
9/23/15 19:30
9/23/15 19:30
9/23/15 19:30
9/23/15 19:30
9/23/15 20:00
9/23/15 20:00
9/23/15 20:00
9/24/15 4:00
9/24/15 4:00
9/24/15 4:00
9/24/15 4:00
9/24/15 4:30
9/24/15 4:30
9/24/15 4:30
9/24/15 4:30


9/27/15 4:00
9/27/15 4:00
9/27/15 4:00
9/27/15 4:30
9/27/15 4:30
9/27/15 4:30
9/27/15 4:30
9/27/15 4:30
9/27/15 5:00
9/27/15 5:00
9/27/15 5:00
9/27/15 5:00
9/27/15 5:30
9/27/15 5:30
9/27/15 5:30
9/27/15 5:30
9/27/15 5:30
9/27/15 6:00
9/27/15 6:00
9/27/15 6:00
9/27/15 6:00
9/27/15 6:30
9/27/15 6:30
9/27/15 6:30
9/27/15 6:30
9/27/15 6:30
9/27/15 7:00
9/27/15 7:00
9/27/15 7:00
9/27/15 7:00
9/27/15 7:30
9/27/15 7:30
9/27/15 7:30
9/27/15 7:30
9/27/15 7:30
9/27/15 8:00
9/27/15 8:00
9/27/15 8:00
9/27/15 8:00
9/27/15 8:30
9/27/15 8:30
9/27/15 8:30
9/27/15 8:30
9/27/15 8:30
9/27/15 9:00
9/27/15 9:00
9/27/15 9:00
9/27/15 9:00
9/27/15 9:30
9/27/15 9:30
9/27/15 9:30
9/27/15 9:30
9/27/15 9:30
9/27/15 10:00
9/27/15 10:00
9/27/15 10:00
9/27/15 10:00
9/27/15 10:00
9/27/15 10:00
9/27/15 10:00
9/27/15 10:30
9/27/15 10:30
9/27/15 10:30
9/27/15 10:30
9/27/15 10:30
9/27/15 11:00
9/27/15 11:00
9/27/15 11:00
9/27/15 11:00
9/27/15 11:00
9/27/15 11:00
9/27/15 11:00
9/27/15 11:30
9/27/15 11:30
9/27/15 11:30
9/2

9/30/15 11:00
9/30/15 11:00
9/30/15 11:00
9/30/15 11:30
9/30/15 11:30
9/30/15 11:30
9/30/15 11:30
9/30/15 11:30
9/30/15 12:00
9/30/15 12:00
9/30/15 12:00
9/30/15 12:00
9/30/15 12:00
9/30/15 12:00
9/30/15 12:00
9/30/15 12:30
9/30/15 12:30
9/30/15 12:30
9/30/15 12:30
9/30/15 12:30
9/30/15 13:00
9/30/15 13:00
9/30/15 13:00
9/30/15 13:00
9/30/15 13:00
9/30/15 13:00
9/30/15 13:00
9/30/15 13:30
9/30/15 13:30
9/30/15 13:30
9/30/15 13:30
9/30/15 13:30
9/30/15 14:00
9/30/15 14:00
9/30/15 14:00
9/30/15 14:00
9/30/15 14:00
9/30/15 14:00
9/30/15 14:00
9/30/15 14:30
9/30/15 14:30
9/30/15 14:30
9/30/15 14:30
9/30/15 14:30
9/30/15 15:00
9/30/15 15:00
9/30/15 15:00
9/30/15 15:00
9/30/15 15:00
9/30/15 15:00
9/30/15 15:00
9/30/15 15:30
9/30/15 15:30
9/30/15 15:30
9/30/15 15:30
9/30/15 15:30
9/30/15 16:00
9/30/15 16:00
9/30/15 16:00
9/30/15 16:00
9/30/15 16:00
9/30/15 16:00
9/30/15 16:00
9/30/15 16:30
9/30/15 16:30
9/30/15 16:30
9/30/15 16:30
9/30/15 16:30
9/30/15 17:00
9/30/15 17:00
9/30/15 17:00
9/30/1

In [28]:

# newFilename = "2/19/15 13:00"
# for i, r in full_df.iloc[:1000].iterrows():
#     if(r['date']==newFilename):
#         statList = list()
#         backIdx = i-1
#         forwardIdx = i+1
#         statList.append(full_df.iloc[i]["PAR"])
#         #XCKD, can change range of days pulled from here by changing size of list
#         # where (size of list - 1) / 2 = number of days in each direction
#         while(len(statList) < 13):
#             # Start iterating backwards and forwards, adding PAR values to stat list as we go
#             # foward and backindex will stop if they reach endpoints, and the other will grow the list
#             # if both endpoints are reached, we stop.
#             if(backIdx > -1):
#                 if(timeMatch(newFilename, full_df.iloc[backIdx]['date'])):
#                     statList.append(full_df.iloc[backIdx]['PAR'])
#             if(forwardIdx < len(full_df)):
#                 if(timeMatch(newFilename, full_df.iloc[forwardIdx]['date'])):
#                     statList.append(full_df.iloc[forwardIdx]['PAR'])
#             backIdx = backIdx - 1
#             forwardIdx = forwardIdx + 1
#             if(forwardIdx >= len(full_df) and backIdx <= -1):
#                 break
#         # If we can find no similar times to take average PAR from then log this in text file and move on
#         if(len(statList) <= 1):
#             with open(resultTextFileName, 'a') as out:
#                 out.write(newFilename + ' has no additional par to take average of...')
#                 break
#         statSeries = pd.Series(statList)
#         #print("erer")
#         descSeries = statSeries.describe()
#         detailsSeries = pd.Series({'newFilename': newFilename,'PAR': full_df.iloc[i]["PAR"], 'PARDIFF':abs(full_df.iloc[i]["PAR"]-descSeries['mean'])})
#         finalInfoSeries = pd.concat([detailsSeries, descSeries])
#         fullInfoList.append(finalInfoSeries)
#         break

# newFilename = "2/19/15 12:00"
# for i, r in full_df.iloc[:1000].iterrows():
#     if(r['date']==newFilename):
#         statList = list()
#         backIdx = i-1
#         forwardIdx = i+1
#         statList.append(full_df.iloc[i]["PAR"])
#         #XCKD, can change range of days pulled from here by changing size of list
#         # where (size of list - 1) / 2 = number of days in each direction
#         while(len(statList) < 13):
#             # Start iterating backwards and forwards, adding PAR values to stat list as we go
#             # foward and backindex will stop if they reach endpoints, and the other will grow the list
#             # if both endpoints are reached, we stop.
#             if(backIdx > -1):
#                 if(timeMatch(newFilename, full_df.iloc[backIdx]['date'])):
#                     statList.append(full_df.iloc[backIdx]['PAR'])
#             if(forwardIdx < len(full_df)):
#                 if(timeMatch(newFilename, full_df.iloc[forwardIdx]['date'])):
#                     statList.append(full_df.iloc[forwardIdx]['PAR'])
#             backIdx = backIdx - 1
#             forwardIdx = forwardIdx + 1
#             if(forwardIdx >= len(full_df) and backIdx <= -1):
#                 break
#         # If we can find no similar times to take average PAR from then log this in text file and move on
#         if(len(statList) <= 1):
#             with open(resultTextFileName, 'a') as out:
#                 out.write(newFilename + ' has no additional par to take average of...')
#                 break
#         statSeries = pd.Series(statList)
#         #print("erer")
#         descSeries = statSeries.describe()
#         detailsSeries = pd.Series({'newFilename': newFilename,'PAR': full_df.iloc[i]["PAR"], 'PARDIFF':abs(full_df.iloc[i]["PAR"]-descSeries['mean'])})
#         finalInfoSeries = pd.concat([detailsSeries, descSeries])
#         fullInfoList.append(finalInfoSeries)
#         break

# newSeries = (pd.concat(fullInfoList, axis=1)).transpose()
# print("ASDASDASD")
# print(newSeries)
# print(type(newSeries))
# newSeries.to_csv(par_csv_save_location+"/"+resultCsvFileName)
# # FIGURE OUT HOW TO COMBINE THESE SERIESES SUCH THAT WE MAY PRINT THEM NICELY

                           FILE        GPP
100  2015_03_18_13_50_bci_t.csv  14.486128
101  2015_03_18_13_55_bci_t.csv  14.306517
102  2015_03_19_12_00_bci_t.csv  20.297166
103  2015_03_19_12_05_bci_t.csv  20.232669
104  2015_03_19_12_10_bci_t.csv  20.168172
..                          ...        ...
994  2015_04_27_12_50_bci_t.csv  29.629066
995  2015_04_27_12_55_bci_t.csv  30.110384
996  2015_04_27_13_00_bci_t.csv  30.591703
997  2015_04_27_13_05_bci_t.csv  29.026826
998  2015_04_27_13_10_bci_t.csv  27.461950

[899 rows x 2 columns]
